In [ ]:
import json
import os

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
%matplotlib inline

In [ ]:
from learning import *

In [ ]:
THOUSAND = 1000
TENTHOUSAND = 10000
HUNDREDTHOUSAND = 100000
MILLION = 1000000

In [ ]:
def get_plot_path(name, ftype, bins):
    return 'agents/{}/{}_dplot_bins_{}_{}.{}'.format(name, name, bins[0], bins[1], ftype)

In [ ]:
def get_plot_path_zoom(name, ftype, bins):
    return 'agents/{}/{}_dplot_zoom_bins_{}_{}.{}'.format(name, name, bins[0], bins[1], ftype)

In [ ]:
name = 'dqn30_5b5_3216_adam_lr0.001_tmu0.01_ml50K_ns5M_eps0.1_a6_b0'

In [ ]:
with open(get_training_path(name), 'r') as f:
    data = json.load(f)

In [ ]:
for k, v in data.items():
    data[k] = np.array(v)

In [ ]:
data

## Simple average and bounds plot ##

Break into chunks of `step` episodes

In [ ]:
avg, lower, upper = [], [], []
rewards = data['episode_reward']
step = 1000
for i in range(0, rewards.shape[0], step):
    sub_rewards = rewards[i:i+step]
    avg.append(np.mean(sub_rewards))
    lower.append(np.amin(sub_rewards))
    upper.append(np.amax(sub_rewards))

In [ ]:
nb_steps = 5*MILLION
nb_episodes = nb_steps/data['nb_episode_steps'][0]
episodes = np.arange(step, nb_episodes+1, step)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

ax.fill_between(episodes, lower, upper, alpha=0.3, label='Bounds', zorder=10)
ax.plot(episodes, avg, color='C1', label='Average', zorder=20)

ax.set_xlabel(r'Episodes', fontsize=18)
ax.set_ylabel(r'Episode reward', fontsize=18)

ax.legend(fontsize=13, loc='upper left', framealpha=1)
    
ax.xaxis.set_major_locator(MultipleLocator(20000))
#ax.xaxis.set_major_formatter('{x:.0f}')
ax.xaxis.set_minor_locator(MultipleLocator(5000))
ax.yaxis.set_major_locator(MultipleLocator(50))
#ax.yaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_minor_locator(MultipleLocator(10))

ax.grid(which='major')
ax.grid(which='minor', alpha=0.5)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(13)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(13)

fig.tight_layout()

fig.savefig(get_plot_path(name, step))

plt.show()

## Density plot ##

In [ ]:
def plot_density(name, bins=(200, 40), episodes=True, fformat=None):
    fig, ax = plt.subplots(figsize=(10, 7))

    bins = bins
    
    if episodes:
        h, xedges, yedges, image = ax.hist2d(data['nb_steps']/50, data['episode_reward'],
                                             bins=bins, cmap=plt.cm.jet, density=False, vmin=0, vmax=120)
    else:
        h, xedges, yedges, image = ax.hist2d(data['nb_steps'], data['episode_reward'],
                                             bins=bins, cmap=plt.cm.jet, density=False, vmin=0, vmax=500)

    if episodes:
        ax.set_xlabel(r'Episode', fontsize=18)
    else:
        ax.set_xlabel(r'Step', fontsize=18)
    ax.set_ylabel(r'Episode reward', fontsize=18)

    #ax.legend(fontsize=13, loc='upper left', framealpha=1)

    #ax.xaxis.set_major_locator(MultipleLocator(1*MILLION))
    #ax.xaxis.set_major_formatter('{x:.0f}')
    #ax.xaxis.set_minor_locator(MultipleLocator(500*THOUSAND))
    #ax.yaxis.set_major_locator(MultipleLocator(50))
    #ax.yaxis.set_major_formatter('{x:.0f}')
    #ax.yaxis.set_minor_locator(MultipleLocator(10))

    #ax.grid(which='major')
    #ax.grid(which='minor', alpha=0.5)

    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(13)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(13)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(image, cax=cax, orientation='vertical')
    cbar.ax.set_ylabel('Frequency', labelpad=5, fontsize=18, rotation=90)
    cbar.ax.tick_params(labelsize=13)

    fig.tight_layout()

    if fformat is not None:
        fig.savefig(get_plot_path(name, fformat, bins))

    plt.show()

In [ ]:
data['nb_steps'].shape

In [ ]:
def plot_density_zoom(name, bins=(200, 40), episodes=True, fformat=None):
    fig, ax = plt.subplots(figsize=(10, 7))

    bins = bins
    
    if episodes:
        h, xedges, yedges, image = ax.hist2d(data['nb_steps'][0:HUNDREDTHOUSAND]/50,
                                             data['episode_reward'][0:HUNDREDTHOUSAND],
                                             bins=bins, cmap=plt.cm.jet, density=False, vmin=0, vmax=120)
    else:
        h, xedges, yedges, image = ax.hist2d(data['nb_steps'][0:HUNDREDTHOUSAND],
                                             data['episode_reward'][0:HUNDREDTHOUSAND],
                                             bins=bins, cmap=plt.cm.jet, density=False, vmin=0, vmax=300)

    if episodes:
        ax.set_xlabel(r'Episode', fontsize=18)
    else:
        ax.set_xlabel(r'Step', fontsize=18)
    ax.set_ylabel(r'Episode reward', fontsize=18)

    #ax.legend(fontsize=13, loc='upper left', framealpha=1)

    #ax.xaxis.set_major_locator(MultipleLocator(1*MILLION))
    #ax.xaxis.set_major_formatter('{x:.0f}')
    #ax.xaxis.set_minor_locator(MultipleLocator(500*THOUSAND))
    #ax.yaxis.set_major_locator(MultipleLocator(50))
    #ax.yaxis.set_major_formatter('{x:.0f}')
    #ax.yaxis.set_minor_locator(MultipleLocator(10))

    #ax.grid(which='major')
    #ax.grid(which='minor', alpha=0.5)

    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(13)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(13)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(image, cax=cax, orientation='vertical')
    cbar.ax.set_ylabel('Frequency', labelpad=5, fontsize=18, rotation=90)
    cbar.ax.tick_params(labelsize=13)

    fig.tight_layout()

    if fformat is not None:
        fig.savefig(get_plot_path_zoom(name, fformat, bins))

    plt.show()

In [ ]:
plot_density(name, bins=(500, 40), episodes=False, fformat='pdf')

In [ ]:
plot_density_zoom(name, bins=(100, 40), episodes=False, fformat='jpg')